In [1]:
import duckdb

con = duckdb.connect()
con.install_extension("substrait")
con.load_extension("substrait")

In [6]:
q_create = "CREATE TABLE bids (id date,bid int,ask int, symbol text);"
q_insert = "INSERT INTO bids VALUES ('2022-01-10', 3, 2, 'gold');"

In [8]:
con.execute(query="DROP TABLE bids;")

In [9]:
con.execute(query=q_create)

In [10]:
con.execute(query=q_insert)

In [11]:
q_select = "SELECT id, bid, ask, symbol FROM bids;"

In [23]:
proto_bytes = con.get_substrait(query=q_select).fetchone()[0] 

In [24]:
proto_bytes

b'\x1a\xa5\x01\x12\xa2\x01\n\x89\x01:\x86\x01\x12V\nT\x126\n\x02id\n\x03bid\n\x03ask\n\x06symbol\x12\x1e\n\x05\x82\x01\x02\x10\x01\n\x04*\x02\x10\x01\n\x04*\x02\x10\x01\n\x07\xb2\x01\x04\x08\x04\x18\x01\x18\x02"\x12\n\x0e\n\x00\n\x02\x08\x01\n\x02\x08\x02\n\x02\x08\x03\x10\x01:\x06\n\x04bids\x1a\x08\x12\x06\n\x02\x12\x00"\x00\x1a\n\x12\x08\n\x04\x12\x02\x08\x01"\x00\x1a\n\x12\x08\n\x04\x12\x02\x08\x02"\x00\x1a\n\x12\x08\n\x04\x12\x02\x08\x03"\x00\x12\x02id\x12\x03bid\x12\x03ask\x12\x06symbol'

In [25]:
query_result = con.from_substrait(proto=proto_bytes)

In [27]:
query_result.to_df()

,id,bid,ask,symbol
0,2022-01-10,3,2,gold


In [31]:
proto_json = con.get_substrait_json(query=q_select).fetchone()[0] 

In [32]:
proto_json

'{"relations":[{"root":{"input":{"project":{"input":{"read":{"baseSchema":{"names":["id","bid","ask","symbol"],"struct":{"types":[{"date":{"nullability":"NULLABILITY_NULLABLE"}},{"i32":{"nullability":"NULLABILITY_NULLABLE"}},{"i32":{"nullability":"NULLABILITY_NULLABLE"}},{"varchar":{"length":4,"nullability":"NULLABILITY_NULLABLE"}}],"nullability":"NULLABILITY_REQUIRED"}},"projection":{"select":{"structItems":[{},{"field":1},{"field":2},{"field":3}]},"maintainSingularStruct":true},"namedTable":{"names":["bids"]}}},"expressions":[{"selection":{"directReference":{"structField":{}},"rootReference":{}}},{"selection":{"directReference":{"structField":{"field":1}},"rootReference":{}}},{"selection":{"directReference":{"structField":{"field":2}},"rootReference":{}}},{"selection":{"directReference":{"structField":{"field":3}},"rootReference":{}}}]}},"names":["id","bid","ask","symbol"]}}]}'